## Kucoin Analysis

In [8]:
from kucoin.client import Client
from pprint import pprint
from configs import *
import pandas as pd
import talib

import cufflinks as cf
cf.set_config_file(theme='pearl',sharing='public',offline=True)

from datetime import datetime, timedelta
time_period = 60 #days
time_start = datetime.timestamp(datetime.now() - timedelta(days=time_period))
#print (time_start)
utc_offset = 8 #hours

symbols = ['DOT-USDT','MATIC-USDT','BTC-USDT']
interval = '1day'

client = Client(api_key, api_secret, api_passphrase)

def get_klines_df(symbol,interval,time_start):
    # get symbol klines
    klines = client.get_kline_data(symbol,interval,start=int(time_start))
    #pprint(klines)

    #Create empty DataFrame with specific column types
    df = pd.DataFrame({'timestamp': pd.Series(dtype='int'),
                    'open': pd.Series(dtype='float'),
                    'close': pd.Series(dtype='float'),
                    'high': pd.Series(dtype='float'),
                    'low': pd.Series(dtype='float'),
                    'volume': pd.Series(dtype='float'),
                    'amount': pd.Series(dtype='float')})

    for item in klines:
        a_series = pd.Series(item, index = df.columns)
        df = df.append(a_series, ignore_index=True)

    df = df.sort_values("timestamp", ascending=True, ignore_index=True)

    df['datetime'] = pd.to_datetime(df['timestamp'],unit='s')
    #df['datetime'] = df['datetime'] + pd.DateOffset(hours=utc_offset)
    df['morning_star'] = talib.CDLMORNINGSTAR(df['open'],df['high'],df['low'],df['close'])
    df['engulfing'] = talib.CDLENGULFING(df['open'],df['high'],df['low'],df['close'])

    return df

df = get_klines_df('LTC-USDT',interval,time_start)

close = df['close']
rsi = talib.RSI(close, timeperiod=14)
upperBB, middleBB, lowerBB = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
upperBBrsi, MiddleBBrsi, lowerBBrsi = talib.BBANDS(rsi, timeperiod=50, nbdevup=2, nbdevdn=2, matype=0)
normrsi = (rsi - lowerBBrsi) / (upperBBrsi - lowerBBrsi)

df['rsi'] = rsi
df['upperBB'] = rsi
df['middleBB'] = rsi
df['lowerBB'] = rsi
df['upperBBrsi'] = rsi
df['MiddleBBrsi'] = rsi
df['lowerBBrsi'] = rsi
df['normrsi'] = rsi

engulfing_days = df[df['engulfing'] != 0]
morningstar_days = df[df['morning_star'] != 0]

In [9]:
df

,timestamp,open,close,high,low,volume,amount,datetime,morning_star,engulfing,rsi,upperBB,middleBB,lowerBB,upperBBrsi,MiddleBBrsi,lowerBBrsi,normrsi
0,1637366400,217.896,226.831,227.76,211.7,157676.09617745,34876144.32368362787,2021-11-20,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1637452800,226.731,221.159,230.805,219.577,157624.91096412,35373580.06805627386,2021-11-21,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1637539200,221.29,209.021,221.757,205.195,184297.76686106,39363325.50463694635,2021-11-22,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1637625600,208.991,216.103,219,203.886,153232.10521698,32319194.26921837851,2021-11-23,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1637712000,216.179,212.312,217.612,205.972,139303.87255279,29388357.58815939545,2021-11-24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1637798400,212.204,222.897,229.653,210.714,134816.7241988,29966881.41820871716,2021-11-25,0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1637884800,223,195.766,224.919,182.317,258868.60259214,52110836.10059167349,2021-11-26,0,-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1637971200,195.716,195.15,200.982,190.591,124994.62482092,24620576.22635487957,2021-11-27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1638057600,195.074,199.519,200,180.652,166017.63893022,31645762.20685816271,2021-11-28,0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1638144000,199.526,205.528,209.544,195.261,117271.55237329,23729438.60505031422,2021-11-29,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
engulfing_days

,timestamp,open,close,high,low,volume,amount,datetime,morning_star,engulfing,rsi,upperBB,middleBB,lowerBB,upperBBrsi,MiddleBBrsi,lowerBBrsi,normrsi
5,1637798400,212.204,222.897,229.653,210.714,134816.7241988,29966881.41820871716,2021-11-25,0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1637884800,223,195.766,224.919,182.317,258868.60259214,52110836.10059167349,2021-11-26,0,-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1638057600,195.074,199.519,200,180.652,166017.63893022,31645762.20685816271,2021-11-28,0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,1639008000,165.079,151.375,167.017,151.137,103797.79447257,16450076.13679386581,2021-12-09,0,-100,25.522603,25.522603,25.522603,25.522603,25.522603,25.522603,25.522603,25.522603
21,1639180800,148.272,158.09,159.504,145.777,122096.46768288,18794921.41567414438,2021-12-11,100,100,31.463434,31.463434,31.463434,31.463434,31.463434,31.463434,31.463434,31.463434
26,1639612800,153.598,148.758,156.664,148.522,75336.33280103,11542115.95462166457,2021-12-16,0,-100,32.249798,32.249798,32.249798,32.249798,32.249798,32.249798,32.249798,32.249798
52,1641859200,126.5,131.546,133.498,125.776,51726.71838265,6721610.03883103791,2022-01-11,0,100,34.431244,34.431244,34.431244,34.431244,34.431244,34.431244,34.431244,34.431244
55,1642118400,136.715,144.741,146.501,135.862,101902.70628421,14485668.38925510946,2022-01-14,0,100,48.735930,48.735930,48.735930,48.735930,48.735930,48.735930,48.735930,48.735930


In [11]:
morningstar_days

,timestamp,open,close,high,low,volume,amount,datetime,morning_star,engulfing,rsi,upperBB,middleBB,lowerBB,upperBBrsi,MiddleBBrsi,lowerBBrsi,normrsi
21,1639180800,148.272,158.09,159.504,145.777,122096.46768288,18794921.41567414438,2021-12-11,100,100,31.463434,31.463434,31.463434,31.463434,31.463434,31.463434,31.463434,31.463434


In [12]:
df.iplot(kind="candle",
                          keys=["open", "high", "how", "close"],
                          rangeslider=False
                          )